# Export to file

[in development]

Exporting to GIS formats such as ESRI Shapefile or GeoTiff requires some libraries which are tricky to install on Windows.

One way is to use Conda-forge

1. Download the [Miniconda installer](https://docs.conda.io/en/latest/miniconda.html)
2. Install GeoPandas: `conda install --channel conda-forge geopandas`

## Export to shapefile

This section requires the `geopandas` package

In [1]:
from mikeio import Dfsu

In [15]:
import geopandas as gpd

In [4]:
dfs = Dfsu("data/oresundHD_run1.dfsu")
dfs

Dfsu2D
Number of elements: 3612
Number of nodes: 2046
Projection: UTM-33
Items:
  0:  Surface elevation <Surface Elevation> (meter)
  1:  Total water depth <Water Depth> (meter)
  2:  U velocity <u velocity component> (meter per sec)
  3:  V velocity <v velocity component> (meter per sec)
Time: 5 steps with dt=86400.0s
      2018-03-07 00:00:00 -- 2018-03-11 00:00:00

In [25]:
ds = dfs.read(items="Surface elevation")
ds

<mikeio.Dataset>
Dimensions: (5, 3612)
Time: 2018-03-07 00:00:00 - 2018-03-11 00:00:00
Items:
  0:  Surface elevation <Surface Elevation> (meter)

In [30]:
maxwl = ds.max(axis=0)["Surface elevation"][0]
maxwl.shape

(3612,)

In [31]:
shp = dfs.to_shapely()
type(shp)

shapely.geometry.multipolygon.MultiPolygon

In [32]:
poly_list = [p for p in shp]

In [39]:
gdf = gpd.GeoDataFrame({'waterlevel': maxwl},geometry=poly_list, crs=dfs.projection_string)

CRSError: Invalid projection: UTM-33: (Internal Proj Error: proj_create: several objects matching this name: Schwarzeck / UTM zone 33S, RGRDC 2005 / UTM zone 33S, Malongo 1987 / UTM zone 33S)

Ouch... The short and smart projection string "UTM-33" is apparently not understood by GeoPandas. Better look it up on the web
https://spatialreference.org/ref/epsg/32433/

In [41]:
gdf = gpd.GeoDataFrame({'waterlevel': maxwl},geometry=poly_list, crs="EPSG:32433")

In [42]:
gdf.head()

,waterlevel,geometry
0,0.321391,"POLYGON ((353122.611 6199631.339, 354250.947 6..."
1,0.254802,"POLYGON ((356811.295 6165979.110, 356926.413 6..."
2,0.210542,"POLYGON ((334193.183 6143454.831, 332951.036 6..."
3,0.213418,"POLYGON ((330871.279 6142677.275, 331831.835 6..."
4,0.208291,"POLYGON ((334327.062 6140906.232, 335381.131 6..."


Export as ESRI Shapefile

In [45]:
gdf.to_file("waterlevel_utm.shp")

Which can be used together with other data sources and customized in QGIS
![](images/qgis_shp.png)

Or GeoJSON, which some might prefer...

In [44]:
gdf.to_file("waterlevel_utm.json")

![](images/notepad_geojson.png)